<h3> Extracting protein sequences' features using ProtBert-BFD pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [15]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm
import pandas as pd
import skimage.measure
from sklearn.manifold import TSNE
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
tqdm.pandas()

<b>2. Load the vocabulary and ProtBert-BFD Model<b>

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False )

In [5]:
model = AutoModel.from_pretrained("Rostlab/prot_bert_bfd")

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<b>3. Load the model into the GPU if avilabile<b>

In [6]:
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer,device=0 ) # device=0 for GPU, -1 for CPU

**4. Load sequences**

Files have to be loaded manually

In [17]:
FILE_KNOTTED = "SPOUT_knotted.csv"
FILE_UNKNOTTED = "Rossmann_unknotted.csv"

In [18]:
df_neg = pd.read_csv(FILE_UNKNOTTED)[["seq"]] #.sample(n=100)
df_neg["label"] = 0

df_pos = pd.read_csv(FILE_KNOTTED, sep=';')[["seq"]] #.sample(n=100)
df_pos["label"] = 1
df_pos

df_merged = pd.concat([df_neg, df_pos], ignore_index=True, sort=False)
# df_merged = df_merged[df_merged['seq'].str.len() < 500]


In [19]:
df_merged['seq'] = df_merged['seq'].apply(lambda sequence: " ".join(re.sub(r"[UZOB]", "X", sequence)))
print(df_merged.head())
print(df_merged.shape)

                                                 seq  label
0  M S V S M R D M L A A G V H F G H Q T R F W N ...      0
1  M A G S E T M E R T V V W F R R D L R I D D N ...      0
2  M E I L S P S P P P S H C P L L R C G G H W E ...      0
3  M I R S V V R S G R T V G R R S S R R L I S Q ...      0
4  M T V A K G E M C S V N D C R F D D N D D E I ...      0
(248444, 2)


<b>5. Extracting sequences' features <b>

In [24]:
def get_embedding(seq):
    embedding = fe(seq)
    features =  np.array(embedding[0][1:len(seq)+1])
    features = skimage.measure.block_reduce(features, (1024, 1), np.average)
    file = open("embedding_features_tmp.csv", 'a')
    file.write(','.join(str(e) for e in features[0]))
    file.write('\n')
    return np.array(features[0], dtype=float)

In [25]:
df_merged['features'] = df_merged['seq'].progress_apply(get_embedding)

100%|██████████| 248444/248444 [3:36:43<00:00, 19.11it/s]   


In [14]:
df_merged['features'].head(n=10)
to_print = pd.DataFrame(df_merged["features"].tolist()).add_prefix("f")
to_print['seq'] = df_merged['seq']
to_print['label'] = df_merged['label']
to_print.to_csv("embedding_features_all.csv", index=False)

**6. Create the visualizatiom**

In [37]:
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(np.array(df_merged['features'].to_list()))
df = pd.DataFrame()
df["y"] = df_merged['label']
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

/home/jovyan/my-conda-envs/knots_ML/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/jovyan/my-conda-envs/knots_ML/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 248444 samples in 0.099s...
[t-SNE] Computed neighbors for 248444 samples in 376.313s...
[t-SNE] Computed conditional probabilities for sample 1000 / 248444
[t-SNE] Computed conditional probabilities for sample 2000 / 248444
[t-SNE] Computed conditional probabilities for sample 3000 / 248444
[t-SNE] Computed conditional probabilities for sample 4000 / 248444
[t-SNE] Computed conditional probabilities for sample 5000 / 248444
[t-SNE] Computed conditional probabilities for sample 6000 / 248444
[t-SNE] Computed conditional probabilities for sample 7000 / 248444
[t-SNE] Computed conditional probabilities for sample 8000 / 248444
[t-SNE] Computed conditional probabilities for sample 9000 / 248444
[t-SNE] Computed conditional probabilities for sample 10000 / 248444
[t-SNE] Computed conditional probabilities for sample 11000 / 248444
[t-SNE] Computed conditional probabilities for sample 12000 / 248444
[t-SNE] Computed conditional proba

In [38]:
sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 2),
                data=df, s=2).set(title="T-SNE projection of embeddings")
plt.savefig("tsne.png", dpi=300)



---



---

